In [92]:
import numpy as np

sample_np = np.random.rand(100, 2)
sample_np.shape

(100, 2)

In [93]:
from sympy import symbols


def polynomial():
    x1 = symbols('x1')
    x2 = symbols('x2')
    return 4*x1**2 + 5*x2**2 + 2*x1*x2 + 3*x1 - 6*x2

In [94]:
from sklearn.preprocessing import PolynomialFeatures
from sympy import Poly, lambdify


def polynomial_regression_gradient_descent(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int):
    # Визначаємо степінь полінома з символьного виразу
    ploinomial_rate = Poly(poloinomial_func).total_degree()

    # Готуємо ціль як значення полінома на даних
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # Узгоджено генеруємо поліноміальні ознаки у train
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # Ключова правка: довжина theta дорівнює кількості колонок у X_poly
    theta = np.zeros(p, dtype=float)

    for _ in range(epochs):
        y_pred = X_poly @ theta          # (m,)
        error = y_pred - y               # (m,)
        gradient = (2.0 / m) * (X_poly.T @ error)  # (p,)
        theta -= learning_rate * gradient

    return theta

In [95]:
def polynomial_regression_SGD(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int):
    # Визначаємо степінь полінома з символьного виразу
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    # Готуємо ціль як значення полінома на даних
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    features_copy = features.copy()
    theta = np.zeros(p, dtype=float)

    for _ in range(epochs):
       np.random.shuffle(features_copy)

In [96]:
def predict(X_input: np.ndarray, poloinomial_func, theta: np.ndarray):
    # Та сама логіка PolynomialFeatures, що і в train
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(X_input)  # Порядок і набір ознак детерміновані degree і n_features
    return X_poly @ theta



In [97]:
def actual_vals(X_input: np.ndarray, poloinomial_func):
    x1, x2 = symbols('x1 x2')
    polynomial = lambdify((x1, x2), poloinomial_func, modules='numpy')
    return polynomial(X_input[:, 0], X_input[:, 1])

In [98]:
test_np = np.random.rand(10, 2)
pol_func = polynomial()
pred = predict(test_np, pol_func, polynomial_regression_gradient_descent(sample_np, pol_func, 0.1, 100000))
pred

array([ 5.55003013,  6.41100536, -0.37786442, -1.37062557, -0.81200282,
        1.37655649,  1.09986384,  1.29739874, -1.35670871,  2.92093165])

In [99]:
actual = actual_vals(test_np, pol_func)
actual

array([ 5.55003013,  6.41100536, -0.37786442, -1.37062557, -0.81200282,
        1.37655649,  1.09986384,  1.29739874, -1.35670871,  2.92093165])